In [ ]:
# Creating and training Machine Learning model
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
lemmatizer = WordNetLemmatizer()
# Load json data from intents.json
intents = json.loads(open('intents.json').read())

words = []
classes =[]
documents = []
ignore_letters = ['?', '!', '.', ',']

#Setting up data

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern) #Split sentence into individual words
        words.extend(word_list) # add words to words list
        documents.append(((word_list), intent['tag'])) # 
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Reduce words to stem
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

#Sort the classes and save to a pickle ifle
classes = sorted(set(classes))
pickle.dump(words,open('words.pk1','wb'))
pickle.dump(classes,open('classes.pk1','wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    if word in word_patterns:
      bag.append(1)
    else:
      bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag,output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

# Neural Network
model = Sequential()
model.add(Dense(128,input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss='categorical_crossentropy',optimizer = sgd, metrics = ['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)

model.save("chatbot_model.model",hist)
print("Done")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
10/10 [==============================] - 1s 2ms/step - loss: 2.1791 - accuracy: 0.1915
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1298 - accuracy: 0.2340
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0830 - accuracy: 0.2766
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0041 - accuracy: 0.2766
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9436 - accuracy: 0.4468
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8989 - accuracy: 0.3617
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8271 - accuracy: 0.4255
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7773 - accuracy: 0.5319
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7160 - accuracy: 0.5106
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5447 - accuracy: 0.5745
Epoch 11/

In [ ]:
# Building the chatbot
from tensorflow.keras.models import load_model

intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pk1','rb'))
classes = pickle.load(open('classes.pk1','rb'))
model = load_model('chatbot_model.model')

#cleaning up sentences
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  result = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]

  result.sort(key = lambda x: x[1], reverse = True)
  return_list = []
  for r in result:
    return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
  return return_list

def get_response(intents_list,intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result


In [ ]:
while True:
  pesan = input("")
  ints = predict_class(pesan)
  res = get_response(ints,intents)
  print(res)

Hi!
Whats up?
Who are you?
Hi my name is Init, I'm here as your companion
Tell me a joke
Why did the chicken commit suicide?
 To get to the other side.
make me laugh
Why did the chicken commit suicide?
 To get to the other side.
help me
Sure, how can I help you?


KeyboardInterrupt: ignored

In [ ]:
# download packages from nltk library
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> ounkt
    Error loading ounkt: Package 'ounkt' not found in index

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
 

True